# Voltage requirements
- 2.5 V: auxiliary power supply
- 1.275 V: bank voltage supply (depending on bank voltage level)
	- e.g.: for DDR3L
- 3.3 V: Vcc / Vcca analog voltage supply / Core voltage supply

# DC-DC converter
Choice: TLV62569DRL, similar to ButterStick, with slightly different 
- capacitor values
- resistor values
- inductor values

## Feedback Resistors

Common values for resistors <= 100 k
- 10k, 12k, 15k, 18k, 20k, 22k, 24k, 27k, 30k, 33k, 39k, 47k, 51k, 56k, 68k, 75k, 82k, 100k

In [1]:
import numpy as np

def get_resistor_ratio(v_out):
	return ((v_out / 0.6) - 1)

resistor_array = np.array([0.470, 0.51, 0.56, 0.68, 0.82, 1, 1.2, 1.5, 1.8, 2, 2.2, 2.4, 2.7, 3, 3.3, 3.6, 3.9, 4.7, 4.99, 5.1, 5.6, 6.2, 
						6.8, 7.5, 8.2, 9.1, 10, 12, 13, 15, 20, 22, 24, 27, 30, 33, 39, 47, 51, 56, 68, 75, 82, 100, 150])
vdd_targets = np.array([1.35, 2.5, 3.3])
r1_arr = [30, 75, 100]
r2_arr = [24, 24, 22]

######### DDR3L
for idx, v_target in enumerate(vdd_targets):
	print(f"---- V = {v_target} ----")
	v_target_ratio = get_resistor_ratio(v_target)
	r2_target_arr = np.round(resistor_array / v_target_ratio, 2)
	r1_target_arr = np.round(resistor_array * v_target_ratio, 2)
	print(f"{v_target} r1/r2: {v_target_ratio}")
	print(f"R2 values {r2_target_arr}")
	print(f"R1 values {r1_target_arr}")

	for r1, r2 in zip(r1_target_arr, r2_target_arr):
		if (r1 in resistor_array):
			print(f"Perfect r1 found: {r1}")
		if (r2 in resistor_array):
			print(f"Perfect r2 found: {r2}")

	r1 = r1_arr[idx]
	r2 = r2_arr[idx]
	vout_actual = 0.6*(1+r1/r2)
	print(f"offset: {((vout_actual - v_target) / v_target) * 100:.2f}%")

---- V = 1.35 ----
1.35 r1/r2: 1.2500000000000004
R2 values [  0.38   0.41   0.45   0.54   0.66   0.8    0.96   1.2    1.44   1.6
   1.76   1.92   2.16   2.4    2.64   2.88   3.12   3.76   3.99   4.08
   4.48   4.96   5.44   6.     6.56   7.28   8.     9.6   10.4   12.
  16.    17.6   19.2   21.6   24.    26.4   31.2   37.6   40.8   44.8
  54.4   60.    65.6   80.   120.  ]
R1 values [  0.59   0.64   0.7    0.85   1.03   1.25   1.5    1.88   2.25   2.5
   2.75   3.     3.38   3.75   4.13   4.5    4.88   5.88   6.24   6.38
   7.     7.75   8.5    9.38  10.25  11.38  12.5   15.    16.25  18.75
  25.    27.5   30.    33.75  37.5   41.25  48.75  58.75  63.75  70.
  85.    93.75 102.5  125.   187.5 ]
Perfect r1 found: 1.5
Perfect r2 found: 1.2
Perfect r1 found: 3.0
Perfect r2 found: 2.4
Perfect r1 found: 15.0
Perfect r2 found: 12.0
Perfect r1 found: 30.0
Perfect r2 found: 24.0
offset: -0.00%
---- V = 2.5 ----
2.5 r1/r2: 3.166666666666667
R2 values [ 0.15  0.16  0.18  0.21  0.26  0.32  0.38 

# DDR3L-power and reference
Use NCP51200

## Electrical characteristics
- Max input voltage: 6 V

## Pins
### REFIN
- Reference input (so feedback should be added here)

## Issue
- No example schematics so not sure 

## Solution
Use the TLVx333 precision amplifier
- input between 1.8 to 5.5 V
- Ouput offset voltage: 2 uV

### Electrical characteristics
- Output current: +5 mA MAX

# Biasing voltage
- Use the LMV321 voltage for offsets

# Power switch

## Features
- Inrush current limit
- A 4 amp max current limit

## Goal
- The power should be supplied from either of the 2 sources.
- The power MAY NOT turn off when adding a new power source.

Use the TPS2121 (seamless switchover)
- So no issues when disconnecting USB and switching to battery power

### Pins
#### Power input
- IN1, IN2: input power pins
	- Note input pin capacitance
	- USB-filtering might be required
- Input capacitance?


#### Status output
- Led required here